<a id="inicio"></a>
<img src="./figs/barra_uclm_esiiab.png" alt="Banner UCLM - ESIIAB" align="right">

<br><br><br>
<h1><font color="#B30033" size=5>Intelligent Systems - Course 2021-2022</font></h1>



<h1><font color="#B30033" size=5>Assignment 2: Heuristic Search Algorithms</font></h1>


<br><br>
<div style="text-align: left">
<font color="#4E70BE" size=3>Lecturers:</font><br>
<ul>
  <li><font color="#4E70BE" size=3>Juan Carlos Alfaro Jiménez</font><br></li>
  <li><font color="#4E70BE" size=3>Guillermo Tomás Fernández Martín</font><br></li>
  <li><font color="#4E70BE" size=3>José Antonio Gámez Martín</font><br></li>
  <li><font color="#4E70BE" size=3>Ismael García Varea</font><br></li>
  <li><font color="#4E70BE" size=3>Luis González Naharro</font><br></li>
  <li><font color="#4E70BE" size=3>Jesús Martínez Gómez</font><br></li>    
</ul>
</div>

<br>

## Introduction
In this assignment we will study and put into practice cost-based and informed, aka heuristic, search algorithms. To do that, some algorithms studied in units 2 and 3 will be implemented and used to solve the same problem we faced with in Assignment 1: maze pathfinding.
We will also analyze and compare the performance of the algorithms by running them over different instances of the problem.

## Problem description
For the sake of completeness we also include the description of the problem in this assignment.

The maze is a grid of size N x M formed by a set of cells, some of which can be occupied by walls, which cannot be crossed. The rest of the cells will be empty and they will represent the free space. For now, the robot can only move horizontally or vertically. In addition, we can have cells with garbage, which must be cleaned by our agent, a vacuum cleaner robot.

The objective of our robot is to clean the whole area as fast as possible. In other words: **find the shortest path to find all the garbage cells in the environment**. In order to implement our robot we have to take into account that: 
- The robot can start in a random cell of the map.
- The robot can move horizontally or vertically in the maze.
- The robot can not cross walls or go beyond the limits of the maze.
- The robot will have to clean all the garbage cells found in the map, which will be automatically cleaned as soon as the robot arrives to those cells.
- For now, all the movements of the robot will have a cost of 1.
- The search will finish once all garbage cells in the map have been cleaned.

## Provided code

In the following we provide you some of the clases, implemented in `Python` that will help you to develop this assignment. 

First, we will import the necessary classes we need from the Python libraries

In [ ]:
import math
import copy
import time

from abc import ABC, abstractmethod
from queue import PriorityQueue

Next, we will import some custom functions from the file `utils.py`. You don't need to modify those functions for the code to work, but feel free to have a look at them if you are curious. This code is identical to the one provided in Assignment 1.

In [ ]:
from utils import *

Finally, we will import some third party libraries. We will use those to display the problem in a graphical environment. In order to do that, we will use the magic functions from jupyter to install the library from inside the notebook

In [ ]:
!pip install ipythonblocks

In [ ]:
from ipythonblocks import BlockGrid
from IPython.display import clear_output

In order to complete the requested search algorithms, we provide you some fundamental classes: 

#### Class `Action`
This class provides the **representation of the actions** that will be performed by the robot. You don't have to modify the code of this class. The possible actions will be: "UP", "DOWN", "RIGHT", "LEFT". This class is identical to the one provided in Assignment and, for the moment, the cost of all actions is kept at the value of 1.0.

In [ ]:
class Action:
    #actions = ["UP", "DOWN", "RIGHT", "LEFT"]

    def __init__(self, move):
        self.move = move

    def __str__(self):
        return f"({self.move})"

    def getCost(self):
        return 1.0

#### Class `State`. 
This class provides the **representation of a state** in the search space. In this problem, a state is defined by the position of the robot and the garbages left in the maze. Note that the maze itself does not need to be part of the state given that it does not change during the search, i.e. walls are fixed during the search. You don't have to modify the code of this class.

The class `State` has an `applyAction` method that, given a valid `Action`, returns a new `State` with the action applied. This class is identical to the one provided in Assignment 1.

In [ ]:
class State:

    def __init__(self, pos, garbage):
        self.pos = pos
        self.garbage = garbage

    # equals method. Returns true if the states are the same. 
    # Used for the hash table comparison, compares if both states are equal
    def __eq__(self, state):
        return self.pos == state.pos and self.garbage == state.garbage

    def __str__(self):
        return f"Position: {self.pos}\nGarbage: {self.garbage}"

    # hash method. Useful to index data structures that uses a hash function 
    #    to index elements, i.e. a Set()
    def __hash__(self):
        h = 0
        for g in self.garbage:
            h += (math.pow(10,3) * (self.pos[0]+1) + (self.pos[1]+2))
        return int(h)

    def applyAction(self, action):
        st = copy.deepcopy(self)
        
        if (action.move == "UP"):
            st.pos = (st.pos[0]-1,st.pos[1])
        elif (action.move == "DOWN"):
            st.pos = (st.pos[0]+1,st.pos[1])
        elif (action.move == "RIGHT"):
            st.pos = (st.pos[0],st.pos[1]+1)
        elif (action.move == "LEFT"):
            st.pos = (st.pos[0],st.pos[1]-1)
        else:
            print("\n*** ERROR ***: Action " + action + "  is not allowed .....\n")
            sys.exit()
            
        # if the new position has garbage clean it
        if (st.pos in st.garbage):
            st.garbage.remove(st.pos)
        
        return st

#### Class `Node`. 
This class provides a **representation of a node** in the search tree/graph. It contains the state it represents, its parent node and the action taken to reach the current node. You don't have to modify this class. 

The class `Node` also has some methods to provide access to its attributes.

**This implementation slightly differs** from the one provided in Assignment 1. Concretely:
- In addition to the `self.gGost` (cost to reach the node), two new attributes are defined: `self.hCost` and `self.fCost`, which are needed to store the heuristic and total cost of a specific node. These attributes should be conveniently assigned during the search process, accoding to the specific search strategy.
- Correspondingly two new methods are included: `getHCost(self)` and `getFCost(self)`
- The `__hash__` method is added, which is a special method useful to index data structures that uses a hash function to index elements.
- The `__lt__` method is added, which is a special method that defines the behaviour of the less-than operator (`<`) for objects of this class. In our case it is defined according to the value of the attribute `self.fCost`.

In [ ]:
class Node:
    def __init__(self, state, parent, action):
        self.state = state # Must be State Class
        self.parent = parent # Must be Node Class
        self.action = action # Must be Action Class
        self.depth = 0
        self.gCost, self.hCost, self.fCost = 0.0, 0.0, 0.0

    def __str__(self):
        return f"depth: {self.gCost} fcost: {self.fCost}, ({self.gCost} + {self.hCost}), state: {self.state}"

    # equal method. Returns true if the states are the same. Used for the hash table comparison
    # compares if both states are equals
    def __eq__(self, other):
        if not isinstance(other, Node):
        # don't attempt to compare against unrelated types
            return NotImplemented
        return self.state == other.state

    # hash method. Useful to index data structures that uses a hash function 
    # to index elements, i.e. a Set()
    def __hash__(self):
        h = 0
        for g in self.state.garbage:
            h += (math.pow(10,3) * (self.state.pos[0]+1) + (self.state.pos[1]+2))
        return int(h)

    def __lt__(self, other):
        if not isinstance(other, Node):
            # don't attempt to compare against unrelated types
            return NotImplemented
        return self.fCost < other.fCost

    def getState(self):
        return self.state

    def getAction(self):
        return self.action
    
    def getParent(self):
        return self.parent

    def getDepth(self):
        return self.depth
    
    def getGCost(self):
        return self.gCost
    
    def getHCost(self):
        return self.hCost

    def getFCost(self):
        return self.fCost
    

## Implementation
In the following we provide you some classes and pieces of code that you will have to complete as a part of this assignment.


#### Class `Problem`
As in Assignment 1, this class provides the **representation of the search problem**. It contains the size of the maze (`rows` and `cols`), the `initialState` and the `maze`. This class can read from a file an instance of the problem to solve, or it can generate a random instance using the size of the grid, the `seed`, the maximum number of divisions (walls) in the maze and the list of garbage cells. You don't have to modify this class. 

The class `Problem` also has some methods to provide access to the initial state, the possible actions and to check if a specific state is the final/goal.

##### Heuristic functions
In this assignment you have to implement a new method, `computeHeuristic(self, st)`, in this class. This method should compute the heuristic cost of a given state, passed as argument. We have included this method in the `Problem` class because typically the computation of the heuristic cost (or distance) highly depends on the problem to solve.

It is mandatory to define and implement at least two different heuristic functions and to empirically compare them. This study should be included in the final report of this assignments, but you have to be sure that both heuristic functions are admissible and consistent. The proof of admissibility and consistency of all proposed heuristic functions also has to be included in the final report.

In [ ]:
class Problem:

    actions = ["UP", "DOWN", "RIGHT", "LEFT"]

    def __init__(self, rows, cols, seed, maxDivisions, garbageCount, heuristic, filename=""):
        self.heuristic = heuristic
        
        if (filename != ""):
            self.rows, self.cols, self.maze = readProblemInstance(filename)
            print('Problem read with size',rows,'x',cols)
        else:
            self.rows = rows
            self.cols = cols
            self.maze = getProblemInstance(rows, cols, maxDivisions, garbageCount, seed)

        self.garbage = []

        for r in range(rows):
            for c in range(cols):
                if self.maze[r][c] == 2:
                    init_point = (r,c)
                elif self.maze[r][c] == 3:
                    self.garbage.append((r,c))

        self.initialState = State(init_point, self.garbage)

    def getInitialState(self):
        return self.initialState

    def getActions(self):
        return self.actions

    def isGoal(self, st):
        '''
        check if the given state is final or not
        '''
        return st.garbage == []
    
    def computeHeuristic(self, st, heuristic="name_of_heuristic"):
        initPos = st.pos
        garbageBis = list(range(len(st.garbage)))
        d = math.inf
        distancia = 0
        
        for i in range(len(garbageBis)):
            if heuristic=="heuristica1":
                distancia = euclidean_distance(initPos,st.garbage[garbageBis[i]])
            elif heuristic =="heuristica2":
                distancia = manhattan_distance(initPos,st.garbage[garbageBis[i]])
            elif heuristic == "None":
                return 0
            else:
                print("Heuristica no implementada")
                return None
            
            if distancia<d:
                d = distancia
        return distancia
        
        '''
        Compute and return the heuristic cost from the state 'st' to the 'goal state'.
        @param st the state
        @param heuristic is the name of the heuristic to use, useful if more than one is implemented 
        '''
        # TODO: implement the computation of the heuristic functions to be used
        # ...
        # end TODO


#### Class `Search`

The `Search` class is an abstract class that contains some attributes:
- The `problem`to solve
- The list of `open` nodes, i.e. nodes in the frontier. This list is undefined given that you have to use a priority queue data structure to implement that list. Remember that nodes will be inserted in order in this list, where the order is defined according to different criteria depending on each specific search algorithm. 
- The list of `closed` nodes to implement the graph search, which is implemented using a `set` data structure.
- The attributes to account for the number of generated and expanded nodes, as well as the maximum size of the open 'list' to know the maximum number of nodes stored simultaneously in memory. Theses attributes are helpful to estimate the time and memory complexity of the algorithms.

This class also provides three methods:
- `insertNode(self, node)`: this is an abstract method that has to be implemented by all (search) classes that inherit from `Search`. You must program this method according to every specific search strategy used.
- `getSuccesors(self, node)`: this method implements the successors function and should return a list with all the valid successors of a given node. You must program this method.
- `doSearch(self)`: this method implements the graph search you have studied in class. You must program this method. It also provides some statistics of the search process.

Please, note that on the contrary to uninformed search, in heuristic search you can use the "content" of the garbage list (i.e. their positions). Also, think about the use of a data structure to conveniently manage the `open` list of nodes.

In [ ]:
class Search(ABC):

    def __init__(self, problem):
        self.problem = problem
        self.open = PriorityQueue()  # TODO: declare self.open as a priority queue COLA DE PRIORIDAD
        self.closed = set()

        self.generatedNodes = 0
        self.expandedNodes = 0
        self.openMaxSize = 0

    @abstractmethod
    def insertNode(self, node):
        pass

    def getSuccesors(self, node):

        suc = []

        # TODO: ADD YOUR CODE HERE
        # ...
        UP = Action("UP")
        DOWN = Action("DOWN")
        LEFT = Action("LEFT")
        RIGHT = Action("RIGHT")

        nodo1UP = Node(node.state.applyAction(UP), node, UP)
        nodo1UP.gCost = node.gCost + UP.getCost()
        nodo1UP.depth = node.depth + 1
        nodo1UP.hCost = self.problem.computeHeuristic(nodo1UP.state,self.problem.heuristic)
        nodo1UP.fCost = nodo1UP.gCost + nodo1UP.hCost

        nodo2DOWN = Node(node.state.applyAction(DOWN), node, DOWN)
        nodo2DOWN.gCost = node.gCost + DOWN.getCost()
        nodo2DOWN.depth = node.depth + 1
        nodo2DOWN.hCost = self.problem.computeHeuristic(nodo2DOWN.state,self.problem.heuristic)
        nodo2DOWN.fCost = nodo2DOWN.gCost + nodo2DOWN.hCost

        nodo3RIGHT = Node(node.state.applyAction(RIGHT), node, RIGHT)
        nodo3RIGHT.gCost = node.gCost + RIGHT.getCost()
        nodo3RIGHT.depth = node.depth + 1
        nodo3RIGHT.hCost = self.problem.computeHeuristic(nodo3RIGHT.state,self.problem.heuristic)
        nodo3RIGHT.fCost = nodo3RIGHT.gCost + nodo3RIGHT.hCost

        nodo4LEFT = Node(node.state.applyAction(LEFT), node, LEFT)
        nodo4LEFT.gCost = node.gCost + LEFT.getCost()
        nodo4LEFT.depth = node.depth + 1
        nodo4LEFT.hCost = self.problem.computeHeuristic(nodo4LEFT.state,self.problem.heuristic)
        nodo4LEFT.fCost = nodo4LEFT.gCost + nodo4LEFT.hCost

        if node.state.pos[0] != 0:
            if self.problem.maze[nodo1UP.state.pos[0]][nodo1UP.state.pos[1]] != 1:
                suc.append(nodo1UP)
                self.generatedNodes += 1
        if node.state.pos[0] != self.problem.rows - 1:
            if self.problem.maze[nodo2DOWN.state.pos[0]][nodo2DOWN.state.pos[1]] != 1:
                suc.append(nodo2DOWN)
                self.generatedNodes += 1
        if node.state.pos[1] != self.problem.cols - 1:
            if self.problem.maze[nodo3RIGHT.state.pos[0]][nodo3RIGHT.state.pos[1]] != 1:
                suc.append(nodo3RIGHT)
                self.generatedNodes += 1
        if node.state.pos[1] != 0:
            if self.problem.maze[nodo4LEFT.state.pos[0]][nodo4LEFT.state.pos[1]] != 1:
                suc.append(nodo4LEFT)
                self.generatedNodes += 1

        return suc

    def doSearch(self):
        totalCost = 0

        # Initial node
        initNode = Node(self.problem.getInitialState(), None, None)
        initNode.gCost = 0.0
        initNode.hCost = self.problem.computeHeuristic(initNode.state,self.problem.heuristic)
        initNode.fCost = initNode.gCost + initNode.hCost
        self.insertNode(initNode)

        self.generatedNodes += 1

        actionSequence = []

        finish = False

        # TODO: add here the main loop of the serch algorithm
        # IMPORTANT: use the method insertNode() to allow for different strategies
        #  ...
        # end TODO
        while not finish:

            if not self.open.not_empty:
                return NotImplemented

            current = self.open.get()[1]
            print(current)

            if finish != self.problem.isGoal(current.state):
                finish = True
                break

            if current.state not in self.closed:
                self.closed.add(current.state)
                s = self.getSuccesors(current)
                
                for i in s:
                    self.insertNode(i)
                    #print(self.open) 
                self.expandedNodes += 1
                
               

        solucion = current
        
        if (finish):
            # TODO: add your code here to collect the solution
            # ...
            #  end TODO
            while current.parent is not None:
                actionSequence.append(current.action)
                current = current.parent
            actionSequence.reverse()

        # TODO: add your code here to print or extract the statistics
        #       for later evaluation of the algorithms
        # ...
        # end TODO

        print("Coste total:", solucion.getGCost())
        print("Profundidad:", solucion.getDepth())

        print("Nodos generados:", self.generatedNodes)
        print("Nodos expandidos:", self.expandedNodes)

        #print("ActionSequence: \n")
        #for i in actionSequence:
        #    print(i)

        #return actionSequence
    

In [ ]:
# Parameter list pregunta 3 (poner "heurisitica2 donde sea necesario")

rows = 12 
cols = 14 
seed = 951 
maxDivisions = 8 
garbageCount = 6
heuristica = "heuristica1"

algorithm = "BestFirst"

path_sol, problem_instance = main(rows, cols, seed, maxDivisions, garbageCount, algorithm, heuristica)

#### Class `UniformCost`, `BestFirst` and `AStar`

These three classes also inherit from `Search` and will implement the uniform cost, best first and $A^*$ search strategies, respectively. You have to implement these three classes.

Despite the `UniformCost` algorithm belongs to the class of non-informed search algorithms, we have included in this assignment because of the similarity in the implementation with the informed search algorithms.

Actually the main difference betweeen these three algorithms is the way in which the function cost for a specific node ($f(n) = g(n) + h(n)$) is computed. Assuming that $g(n)$ is the real accumulated cost from the **initial state** to `n.getState()` and that $h(n)$ is the heuristic cost from `n.getState()` state to the **goal state**, $f(n)$ is computed as:

- Uniform cost: $f(n) = g(n)$
- Best First: $f(n) = h(n)$
- A$^*$: $f(n) = g(n) + h(n)$

As in Assignment 1, once the `getSuccessors(self,node)` and `doSearch(self)` methods have been implemented in the parent class, we only have to implement the `insertNode(self, node)` method, which will insert the `node`into the `self.open` list of nodes according to the corresponding values of the cost function.

In [ ]:
class UniformCost(Search):
    def insertNode(self, node):
        self.open.put((node.gCost, node))

In [ ]:
class BestFirst(Search):
    def insertNode(self, node):
        self.open.put((node.hCost, node))

In [ ]:
class AStar(Search):
    def insertNode(self, node):
        self.open.put((node.fCost, node))

#### Classes `DepthFirst`, `BreadthFirst` and `DepthLimited`

These classes, implemented in Assignment 1, also can inherit from `Search` and implement the different non-informed search techniques already studied. These classes can also be used in this assigment in order to compare results and behaviours with informed search algorithms. In that case you will also have to define 
the corresponding `insertNode(self, node)` functions using the priority queue data structure and think about what value should be used for every algorithm to insert elements in the correct order to maintain the correct behaviour of every algorithm.

In [ ]:
class DepthFirst(Search):
    def insertNode(self, node):
        self.open.put((-(self.open.qsize()), node))

class BreadthFirst(Search):
    def insertNode(self, node):
        self.open.put((self.open.qsize(), node))

class DepthLimited(Search):
    def insertNode(self, node):
        self.open.put((-(self.open.qsize()), node))

#### The `main` function

Next, we provide you the `main` function that creates the problem and solves it using the search algorithm provided. This method should be used afterwards to carry out the experimentation to study the behaviour of the implemented algorithms for different values of the parameters provided (size of the maze, maximum number of walls, number of garbage cells, and algorithm).

In [ ]:
def main(rows, cols, seed, maxDivisions, garbageCount, algorithm, heuristic, configFile=""):
    problem = Problem(rows, cols, seed, maxDivisions, garbageCount, heuristic, configFile)

    print("$ python assignment1 " + str(problem.rows) + " " + str(problem.cols) + " " + str(seed) + " " + str(maxDivisions) + " " + str(garbageCount) + " " + algorithm + " " + configFile + "\n")
    print("Problem instance:")
    printMaze(problem.maze)
    print("")

    search = None

    if algorithm == "BreadthFirst":
        search = BreadthFirst(problem)
    elif algorithm == "DepthFirst":
        search = DepthFirst(problem)
    elif algorithm == "DepthLimited":
        search = DepthLimited(problem)
    elif algorithm == "UniformCost":
        search = UniformCost(problem)
    elif algorithm == "BestFirst":
        search = BestFirst(problem)
    elif algorithm == "AStar":
        search = AStar(problem)
    else:
        raise Exception

    time_start = time.perf_counter()
    path = search.doSearch()
    time_end = time.perf_counter()
    print("")
    print("Elapsed time: " + str(time_end - time_start) + " seconds")

    return path, problem

#### Test your code

Here you have a piece of code to test your implementation. For example, the code to execute a random search technique can be:

In [ ]:
# Parameter list

rows = 20
cols = 20
seed = 123
maxDivisions = 15
garbageCount = 3
heuristica = "heuristica1"

algorithm = "BestFirst"

path_sol, problem_instance = main(rows, cols, seed, maxDivisions, garbageCount, algorithm, heuristica)

In [ ]:
# Parameter list pregunta 1

rows = 12 
cols = 14 
seed =  951 
maxDivisions = 8 
garbageCount = 6 
heuristica = "None"

algorithm = "UniformCost"

# path_sol, problem_instance = main(rows, cols, seed, maxDivisions, garbageCount, algorithm, heuristica)

In [ ]:
# Parameter list pregunta 2 (poner "heurisitica1 donde sea necesario")

rows = 12 
cols = 14 
seed = 951 
maxDivisions = 8 
garbageCount = 6
heuristica = "heuristica1"

algorithm = "BestFirst"

# path_sol, problem_instance = main(rows, cols, seed, maxDivisions, garbageCount, algorithm, heuristica)

In [ ]:
# Parameter list pregunta 3 (poner "heurisitica2 donde sea necesario")

rows = 12 
cols = 14 
seed = 951 
maxDivisions = 8 
garbageCount = 6
heuristica = "heuristica2"

algorithm = "AStar"

path_sol, problem_instance = main(rows, cols, seed, maxDivisions, garbageCount, algorithm, heuristica)

In [ ]:
# Parameter list pregunta 10(BestFirst) (poner "heurisitica2 donde sea necesario")

rows = 25 
cols = 25 
seed = 951
maxDivisions = 8 
garbageCount = 25
heuristica = "heuristica2"

search = "BestFirst"

# path_sol, problem_instance = main(rows, cols, seed, maxDivisions, garbageCount, algorithm, heuristica)

In [ ]:
# Parameter list pregunta 10(AStar) (poner "heurisitica2 donde sea necesario")

rows = 25 
cols = 25 
seed = 951
maxDivisions = 8 
garbageCount = 25
heuristica = "heuristica2"

search ="AStar" 

# path_sol, problem_instance = main(rows, cols, seed, maxDivisions, garbageCount, algorithm, heuristica)

#### Printing the result

Here we provide you some code to display the maze and the path carried out by the robot (the green cell) to solve the instance of the problem. Walls are represented as black cells, and garbage with brown cells.

In [ ]:
def render_maze(grid, maze, garbage):
    height, width = len(maze), len(maze[0])
    # Render maze
    for i in range(width):
        for j in range(height):
            grid[j,i] = (200,200,200) if maze[j][i] in [0,2,3] else (0,0,0)

    # Render garbage in maze
    for g in garbage:
        grid[g[0],g[1]] = (139,69,19)
        
def find_agent(maze, width, height):
    for i in range(width):
        for j in range(height):
            if (maze[j][i] == 2):
                return (i,j)

def render_path(path, maze, garbage):
    height, width = len(maze), len(maze[0])
    solution_grid = BlockGrid(width, height, fill=(200, 200, 200))
    
    movementDict = {'DOWN':(0,1), 'UP':(0,-1), 'LEFT':(-1,0), 'RIGHT':(1,0)}
    agentPos = find_agent(maze, width, height)
    garbageRender = copy.deepcopy(garbage)
    
    # Initial position rendering
    render_maze(solution_grid, maze, garbageRender)
    solution_grid[agentPos[1],agentPos[0]] = (0, 255, 0)
    solution_grid.show()
    clear_output(wait=True)
    time.sleep(0.1)
    
    for action in path:
        # Update agent position
        agentPos = (agentPos[0] + movementDict[action.move][0], agentPos[1] + movementDict[action.move][1])
        
        # Update garbage list
        if ((agentPos[1], agentPos[0]) in garbageRender):
            garbageRender.remove((agentPos[1], agentPos[0]))
        
        # Render maze
        render_maze(solution_grid, maze, garbageRender)
        
        # Render agent, and update its position and the garbage list
        solution_grid[agentPos[1],agentPos[0]] = (0,255,0)
        
        solution_grid.show()
        clear_output(wait=True)
        time.sleep(0.2)

In [ ]:
render_path(path_sol, problem_instance.maze, problem_instance.garbage)

You can easily try different instances of the problem just by changing the parameters when you call both the `main(...)` function and the `render_path(...)` function:

In [ ]:
path_sol, problem_instance = main(15, 15, 2021, 15, 6, 'BestFirst')

In [ ]:
render_path(path_sol, problem_instance.maze, problem_instance.garbage)

## Experimental results

Once the algorithms have been implemented, you must study their performance. In order to do that, you must compare the quality of the solutions obtained, as well as the number of expanded nodes for instances of different maze sizes, number of walls and number of garbage cells.

Please, use new cells to insert code to carry out the experimental results and study of the algorithms.

In [ ]:
rows = 14
cols = 16
seed = 945
maxDivisions = 8
garbageCount = 6
# path_sol, problem_instance = main(rows, cols, seed, maxDivisions, garbageCount, "AStar", "heuristica2")